In [1]:
# To automatically reload
%reload_ext autoreload
%autoreload 2

In [18]:
# import required libraries
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as img
import os
import numpy as np


In [19]:
!ls dataset/

lip_train_set.csv  README.md	   train_set	 val_set
lip_val_set.csv    train_data.ftr  val_data.ftr  vis_annotation.py


In [5]:
# csv file line format:
# ImageID_PersonId.jpg,x1,y1,v1,x2,y2,v2,...x16,y16,v16
# Note: x,y, is the annotation label in (column, row),
#       v stands for visuable
        
# Joint order:
#     1,R_Ankle
#     2,R_Knee
#     3,R_Hip
#     4,L_Hip
#     5,L_Knee
#     6,L_Ankle
#     7,B_Pelvis
#     8,B_Spine
#     9,B_Neck
#     10,B_Head
#     11,R_Wrist
#     12,R_Elbow
#     13,R_Shoulder
#     14,L_Shoulder
#     15,L_Elbow
#     16,L_Wrist
col_names = ['ImageID_PersonId', 'R_Ankle_x', 'R_Ankle_y', 'R_Ankle_v',
            'R_Knee_x', 'R_Knee_y', 'R_Knee_v',
            'R_Hip_x', 'R_Hip_y', 'R_Hip_v',
            'L_Hip_x', 'L_Hip_y', 'L_Hip_v',
            'L_Knee_x', 'L_Knee_y', 'L_Knee_v',
            'L_Ankle_x', 'L_Ankle_y', 'L_Ankle_v',
            'B_Pelvis_x', 'B_Pelvis_y', 'B_Pelvis_v',
            'B_Spine_x', 'B_Spine_y', 'B_Spine_v',
            'B_Neck_x', 'B_Neck_y', 'B_Neck_v',
            'B_Head_x', 'B_Head_y', 'B_Head_v',
            'R_Wrist_x', 'R_Wrist_y', 'R_Wrist_v',
            'R_Elbow_x', 'R_Elbow_y', 'R_Elbow_v',
            'R_Shoulder_x', 'R_Shoulder_y', 'R_Shoulder_v',
            'L_Shoulder_x', 'L_Shoulder_y', 'L_Shoulder_v',
            'L_Elbow_x', 'L_Elbow_y', 'L_Elbow_v',
            'L_Wrist_x', 'L_Wrist_y', 'L_Wrist_v']

In [20]:
# read csv containing image name and corresponding keypoints
df = pd.read_csv('dataset/lip_val_set.csv', names=col_names)
df.head()

,ImageID_PersonId,R_Ankle_x,R_Ankle_y,R_Ankle_v,R_Knee_x,R_Knee_y,R_Knee_v,R_Hip_x,R_Hip_y,R_Hip_v,...,R_Shoulder_v,L_Shoulder_x,L_Shoulder_y,L_Shoulder_v,L_Elbow_x,L_Elbow_y,L_Elbow_v,L_Wrist_x,L_Wrist_y,L_Wrist_v
0,100034_483681.jpg,153.0,351.0,0.0,122.0,284.0,0.0,NaN,NaN,NaN,...,1.0,93.0,98.0,0.0,121.0,157.0,0.0,81.0,208.0,0.0
1,10005_205677.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,112.0,93.0,0.0,127.0,169.0,0.0,105.0,238.0,0.0
2,100142_449784.jpg,29.0,468.0,0.0,65.0,384.0,0.0,72.0,267.0,0.0,...,0.0,138.0,100.0,0.0,160.0,149.0,1.0,168.0,191.0,0.0
3,10014_1211482.jpg,54.0,242.0,0.0,29.0,193.0,0.0,13.0,142.0,0.0,...,0.0,82.0,61.0,0.0,92.0,103.0,0.0,83.0,124.0,0.0
4,10024_490664.jpg,47.0,243.0,0.0,47.0,185.0,0.0,34.0,125.0,0.0,...,0.0,30.0,57.0,0.0,10.0,99.0,0.0,10.0,111.0,0.0


## Data clean up

In [21]:
def replaceNAN(df, idx, val):
    """ replace the NaN entries by val"""
    if np.isnan(df.iloc[idx]):
        df.iloc[idx - 1] = val
        df.iloc[idx - 2] = val
        
    return df

In [22]:
# change all nan keypoint values to -1 
for i in range(3, df.shape[1], 3):
    df = df.apply(replaceNAN, axis=1, idx=i, val=-1)

In [23]:
df.head()

,ImageID_PersonId,R_Ankle_x,R_Ankle_y,R_Ankle_v,R_Knee_x,R_Knee_y,R_Knee_v,R_Hip_x,R_Hip_y,R_Hip_v,...,R_Shoulder_v,L_Shoulder_x,L_Shoulder_y,L_Shoulder_v,L_Elbow_x,L_Elbow_y,L_Elbow_v,L_Wrist_x,L_Wrist_y,L_Wrist_v
0,100034_483681.jpg,153.0,351.0,0.0,122.0,284.0,0.0,-1.0,-1.0,NaN,...,1.0,93.0,98.0,0.0,121.0,157.0,0.0,81.0,208.0,0.0
1,10005_205677.jpg,-1.0,-1.0,NaN,-1.0,-1.0,NaN,-1.0,-1.0,NaN,...,0.0,112.0,93.0,0.0,127.0,169.0,0.0,105.0,238.0,0.0
2,100142_449784.jpg,29.0,468.0,0.0,65.0,384.0,0.0,72.0,267.0,0.0,...,0.0,138.0,100.0,0.0,160.0,149.0,1.0,168.0,191.0,0.0
3,10014_1211482.jpg,54.0,242.0,0.0,29.0,193.0,0.0,13.0,142.0,0.0,...,0.0,82.0,61.0,0.0,92.0,103.0,0.0,83.0,124.0,0.0
4,10024_490664.jpg,47.0,243.0,0.0,47.0,185.0,0.0,34.0,125.0,0.0,...,0.0,30.0,57.0,0.0,10.0,99.0,0.0,10.0,111.0,0.0


In [24]:
# remove all columns otherthan keypoints (names ending with 'v')
df.drop(df.columns[range(3, df.shape[1], 3)], axis=1, inplace=True)
df.head()

,ImageID_PersonId,R_Ankle_x,R_Ankle_y,R_Knee_x,R_Knee_y,R_Hip_x,R_Hip_y,L_Hip_x,L_Hip_y,L_Knee_x,...,R_Elbow_x,R_Elbow_y,R_Shoulder_x,R_Shoulder_y,L_Shoulder_x,L_Shoulder_y,L_Elbow_x,L_Elbow_y,L_Wrist_x,L_Wrist_y
0,100034_483681.jpg,153.0,351.0,122.0,284.0,-1.0,-1.0,152.0,198.0,79.0,...,96.0,162.0,74.0,95.0,93.0,98.0,121.0,157.0,81.0,208.0
1,10005_205677.jpg,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,15.0,164.0,20.0,102.0,112.0,93.0,127.0,169.0,105.0,238.0
2,100142_449784.jpg,29.0,468.0,65.0,384.0,72.0,267.0,146.0,274.0,138.0,...,28.0,196.0,33.0,111.0,138.0,100.0,160.0,149.0,168.0,191.0
3,10014_1211482.jpg,54.0,242.0,29.0,193.0,13.0,142.0,61.0,134.0,61.0,...,-1.0,-1.0,2.0,53.0,82.0,61.0,92.0,103.0,83.0,124.0
4,10024_490664.jpg,47.0,243.0,47.0,185.0,34.0,125.0,80.0,121.0,73.0,...,109.0,83.0,87.0,53.0,30.0,57.0,10.0,99.0,10.0,111.0


In [25]:
# saving the dataframe in feather format (because it is fast)
df.to_feather('dataset/val_data.ftr')

## Visualization

In [14]:
val_df = pd.read_feather('dataset/val_data.ftr')
val_df.head()

,ImageID_PersonId,R_Ankle_x,R_Ankle_y,R_Knee_x,R_Knee_y,R_Hip_x,R_Hip_y,L_Hip_x,L_Hip_y,L_Knee_x,...,R_Elbow_x,R_Elbow_y,R_Shoulder_x,R_Shoulder_y,L_Shoulder_x,L_Shoulder_y,L_Elbow_x,L_Elbow_y,L_Wrist_x,L_Wrist_y
0,100034_483681.jpg,153.0,351.0,122.0,284.0,-1.0,-1.0,152.0,198.0,79.0,...,96.0,162.0,74.0,95.0,93.0,98.0,121.0,157.0,81.0,208.0
1,10005_205677.jpg,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,15.0,164.0,20.0,102.0,112.0,93.0,127.0,169.0,105.0,238.0
2,100142_449784.jpg,29.0,468.0,65.0,384.0,72.0,267.0,146.0,274.0,138.0,...,28.0,196.0,33.0,111.0,138.0,100.0,160.0,149.0,168.0,191.0
3,10014_1211482.jpg,54.0,242.0,29.0,193.0,13.0,142.0,61.0,134.0,61.0,...,-1.0,-1.0,2.0,53.0,82.0,61.0,92.0,103.0,83.0,124.0
4,10024_490664.jpg,47.0,243.0,47.0,185.0,34.0,125.0,80.0,121.0,73.0,...,109.0,83.0,87.0,53.0,30.0,57.0,10.0,99.0,10.0,111.0


In [15]:
val_df.iloc[1].values

array(['10005_205677.jpg', -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0,
       -1.0, -1.0, -1.0, -1.0, 63.0, 236.0, 57.0, 128.0, 60.0, 87.0, 13.0,
       36.0, 18.0, 214.0, 15.0, 164.0, 20.0, 102.0, 112.0, 93.0, 127.0,
       169.0, 105.0, 238.0], dtype=object)

In [16]:
def plot_joint(rec, img_folder):
    ''' Drawing joints and bones on images '''
    img_name = os.path.join(img_folder, rec[0])
    print('Image at: ' + img_name)

    img = cv2.imread(img_name)
    bombs = [[0,1],[1,2]
            ,[3,4],[4,5]
            ,[6,7],[7,8],[8,9]
            ,[10,11],[11,12]
            ,[13,14],[14,15] ]
    colors = [(255,0,0),(255,0,0),
              (0,255,0),(0,255,0),
              (0,0,255),(0,0,255),(0,0,255),
              (128,128,0),(128,128,0),
              (128,0,128),(128,0,128)]

    for b_id in range(len(bombs)):
        b = bombs[b_id]
        color = colors[b_id]
        x1 = rec[ b[0] * 2 + 1]
        y1 = rec[ b[0] * 2 + 2]

        x2 = rec[ b[1] * 2 + 1]
        y2 = rec[ b[1] * 2 + 2]

        if x1 > 0 and x2 > 0 and y1 > 0 and y2 > 0:
            img = cv2.line(img, (int(x1),int(y1)), (int(x2),int(y2)), color, 4) 
        elif x1 > 0 and y1 > 0:
            img = cv2.circle(img, (int(x1), int(y1)), 5, color, 4) 
        elif x2 > 0 and y2 > 0:
            img = cv2.circle(img, (int(x2), int(y2)), 5, color, 4)
    
    cv2.imshow('Keypoints', img)
    cv2.waitKey(0)
        
    cv2.destroyAllWindows()


In [17]:
plot_joint(val_df.iloc[100].values , 'dataset/val_set/')

Image at: dataset/val_set/10710_428800.jpg


## Creating Pytorch Dataset

In [2]:
from utils.LIPDataset import LIPDataset

In [3]:
PIL_dataset = LIPDataset('train_data.ftr', 'dataset/train_set/')

In [4]:
sample = PIL_dataset[1722]

In [5]:
from utils.LIPDataset import plot_data

In [6]:
plot_data(sample)

## Apply transformations

In [6]:
from utils.LIPDataset import Resize

In [8]:
sample['image'].shape, sample['keypoints']

((152, 248, 3),
 array([-1.0, -1.0, 94.0, 108.0, 238.0, 52.0, 167.0, 103.0, 62.0, 122.0,
        -1.0, -1.0, 197.0, 81.0, 191.0, 6.0, -1.0, -1.0, -1.0, -1.0, 100.0,
        57.0, 137.0, 29.0, -1.0, -1.0, -1.0, -1.0, 88.0, 41.0, 23.0, 73.0],
       dtype=object))

In [9]:
plot_data(sample)

In [10]:
# initialize a resize transform
resize = Resize((256, 256))

In [11]:
# line by line profiling
%load_ext line_profiler
# lprun -f

In [12]:
# performing tfm and examining the results
tfm_sample = resize(sample)

In [13]:
tfm_sample['image'].shape, tfm_sample['keypoints']

((256, 256, 3),
 [-1.032258064516129,
  -1.6842105263157894,
  97.03225806451613,
  181.89473684210526,
  245.6774193548387,
  87.57894736842104,
  172.38709677419354,
  173.4736842105263,
  64.0,
  205.4736842105263,
  -1.032258064516129,
  -1.6842105263157894,
  203.3548387096774,
  136.42105263157893,
  197.16129032258064,
  10.105263157894736,
  -1.032258064516129,
  -1.6842105263157894,
  -1.032258064516129,
  -1.6842105263157894,
  103.2258064516129,
  96.0,
  141.41935483870967,
  48.84210526315789,
  -1.032258064516129,
  -1.6842105263157894,
  -1.032258064516129,
  -1.6842105263157894,
  90.83870967741936,
  69.05263157894737,
  23.741935483870968,
  122.94736842105263])

In [19]:
plot_data(tfm_sample)

In [16]:
from utils.LIPDataset import RandomCrop

In [25]:
# initialize RandomCrop
random_crop = RandomCrop(224)

In [26]:
tfm_sample = random_crop(sample)

In [27]:
plot_data(tfm_sample)

In [38]:
from utils.LIPDataset import ToTensor

In [39]:
to_tensor = ToTensor()

In [40]:
tfm_sample = to_tensor(sample)

In [44]:
tfm_sample['image'].size(), tfm_sample['keypoints'].size()

(torch.Size([3, 372, 277]), torch.Size([32]))

In [36]:
from utils.LIPDataset import RandomHorizontalFlip

In [37]:
rand_flip = RandomHorizontalFlip(.6)

In [42]:
tfm_sample = rand_flip(sample)
plot_data(tfm_sample)

## Final test of combined transforms

In [2]:
## defining a complete data transform
from torchvision import transforms
from utils.LIPDataset import Resize, RandomCrop, RandomHorizontalFlip, ToTensor

In [3]:
data_transform = transforms.Compose([Resize(256),
                                    RandomCrop(224),
                                    RandomHorizontalFlip(0.7)])

In [6]:
from utils.LIPDataset import LIPDataset, plot_data

In [5]:
dataset = LIPDataset('train_data.ftr',
                     'dataset/train_set/',
                    transform=data_transform)

In [10]:
plot_data(dataset[100])